In [17]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
import torch.nn as nn
from torchvision.utils import save_image
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
import copy
import torch.nn.functional as F

from pathlib import Path
from PIL import Image
from torchvision import transforms
import telebot
bot = telebot.TeleBot('5676488946:AAHB6BtC1S1vEiEjm6zoGQJ_qtq_lGH4KDA')
%matplotlib inline


%matplotlib inline

In [18]:
class Shuffle_Block(nn.Module):
  def __init__(self, channels = 64):
    super().__init__()
    self.block = nn.Sequential(
        nn.Conv2d(in_channels = channels, out_channels =4*channels, kernel_size = 3, stride = 1, padding = 1),
        nn.PixelShuffle(upscale_factor = 2),
        nn.PReLU()
    )

  def forward(self, x):
    return self.block(x) # [B, channels, W*2, H*2]

In [19]:
class Residual_Block(nn.Module):
  def __init__(self, channels = 64):
    super().__init__()
    self.block = nn.Sequential(
       nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, stride = 1, padding = 1),
       nn.BatchNorm2d(num_features = channels),
       nn.PReLU(),

       nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, stride = 1, padding = 1),
       nn.BatchNorm2d(num_features = channels),
    )

  def forward(self, x):
    return x+self.block(x) # elementwise summ

In [20]:
class Generator(nn.Module):
  def __init__(self, channels = 64, num_residual_blocks = 10, num_shuffle_blocks = 2 ):
    super().__init__()

    self.in_layer = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = channels, kernel_size = 9, stride = 1, padding = 4),
        nn.PReLU(),
    )

    self.mid_layer = nn.Sequential(
        nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(channels)
    )

    self.end_layer = nn.Sequential(
        nn.Conv2d(in_channels = channels, out_channels = 3, kernel_size = 9, stride = 1, padding = 4),
        nn.Tanh()
    )
    
    self.residual_blocks = []
    for _ in range(num_residual_blocks):
      self.residual_blocks+=[Residual_Block(channels = channels)]
    
    self.shuffle_blocks = []
    for _ in range(num_shuffle_blocks):
      self.shuffle_blocks+=[Shuffle_Block(channels = channels)]

    self.residual_blocks = nn.Sequential(*self.residual_blocks)
    self.shuffle_blocks = nn.Sequential(*self.shuffle_blocks)


  def forward(self, x):
    x = self.in_layer(x)
    x = x + self.residual_blocks(x)
    x = self.shuffle_blocks(x)
    return self.end_layer(x)

In [21]:
class img_processing(nn.Module):
    def __init__(self):
        super().__init__()

        self.transform1=  tt.Compose([
            tt.ToTensor(),
            tt.ConvertImageDtype(torch.float)
        ])

        self.transform2= tt.Compose([
            tt.ToPILImage()
        ])
        
        self.model= Generator(channels= 64, num_residual_blocks= 16, num_shuffle_blocks= 2)
        self.model=torch.load(r'C:\Users\xiaom\Desktop\git\Project\model_SRgenerator_v3.pt')
        self.model.eval()
    
    def forward(self,img):
        img= self.transform1(img)
        size= img.shape
        img= img.view(1,size[0], size[1],size[2])
        img= (self.model(img)+1)/2
        img= self.transform2(torch.squeeze(img))
        return img

processing= img_processing()

In [22]:
@bot.message_handler(commands=["start"])
def start(m):
    chat_id= m.chat.id
    text= m.text
    msg= bot.send_message(m.chat.id,'HightResBot- бот для учучшения качества фотографий формата jpg. Формат png алгоритм обрабатывает хуже.\
    \n\nПо всем вопросам пишите @vldmr_sp.\
    \n\nЕсли вы хотите узнать дополнительную информацию. /help\
    \n\nХотите продолжить? /yes'
    )


@bot.message_handler(commands=["help"])
def help(m):
    chat_id= m.chat.id
    text= m.text
    msg= bot.send_message(chat_id, \
    'Бот использует архетектуру генеративно-состязательной нейросети для обработки фотографий формата jpg.\
    \nКачество может улучшиться вплоть до 4x.\
    \n\nОбьекты png имеют сжатие, на которых нейросеть не училась.\
    \n\nБот не сохраняет фотографии и не передает информацию третим лицам\
    \n\nХотите продолжить? /yes'
    )    

In [23]:
from tkinter import EXCEPTION


photo_list= []

@bot.message_handler(content_types=['text'])
def handle_text(m):
    chat_id = m.chat.id
    text = m.text
    if text.strip().lower() not in ['/да','/yes','/da','/if']:
        msg= bot.send_message(chat_id, 'Пожалуйста, повторите попытку.')
        bot.register_next_step_handler(msg, handle_text)
    else: 
        msg= bot.send_message(chat_id, 'Хорошо, продолжаем...\nЗагрузите фотографию.')
        bot.register_next_step_handler(msg, photo_id)

@bot.message_handler(content_types=["photo"])
def photo_id(m):
    chat_id= m.chat.id
    if m.text == '/done':
        return
    else:
            file_info = bot.get_file(m.photo[len(m.photo) - 1].file_id)
            downloaded_file = bot.download_file(file_info.file_path)
            src =m.photo[1].file_id
            with open(src, 'wb') as new_file:
                new_file.write(downloaded_file)
                img = processing(Image.open(src))
                bot.send_photo(chat_id, photo=img)
            
            os.remove(src)
    
bot.polling(none_stop=True, interval=0)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 314572800 bytes.